In [1]:
from newspaper import Article
import nltk


In [ ]:
# Download required nltk data (only needed once)
nltk.download('punkt')

# Initialize an Article object with a URL
url = "https://example.com/news-article"
article = Article(url)

# Download and parse the article
article.download()
article.parse()

# Extract basic information
print(f"Title: {article.title}")
print(f"Authors: {article.authors}")
print(f"Publish Date: {article.publish_date}")
print(f"Top Image: {article.top_image}")

# Extract the main text content
print(f"Text: {article.text}")

# Perform natural language processing
article.nlp()

# Get keywords and summary
print(f"Keywords: {article.keywords}")
print(f"Summary: {article.summary}")

# Get other metadata
print(f"Meta Data: {article.meta_data}")

In [3]:
import requests
from bs4 import BeautifulSoup


In [19]:

def scrape_vot_article(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract title
        title = soup.find('h1', class_='tdb-title-text')
        title_text = title.text.strip() if title else "Title not found"

        # Extracting Meta Data
        meta_data_body = soup.find('div', class_="vc_column_inner tdi_85 wpb_column vc_column_container tdc-inner-column td-pb-span6")
        author_name = meta_data_body.find('a', class_="tdb-author-name").get_text()
        date_time_meta = meta_data_body.find('time', class_="entry-date updated td-module-date").get_text()

        # getting tag meta data 
        tag_meta = soup.find('ul', class_='tdb-tags')
        tag_meta = tag_meta.select('li a')
        # print(tag_meta)
        # store the tags in a list
        tag_list_meta = [tag.text for tag in tag_meta]
        

        
        website_meta = url

        meta_data = {
            "Author": author_name,
            "Date": date_time_meta,
            "Tags": tag_list_meta,
            "URL": website_meta
        }

        # Extract body content
        body = soup.find('div', class_='td_block_wrap tdb_single_content tdi_100 td-pb-border-top td_block_template_1 td-post-content tagdiv-type')
        if body:

            # Extracting all <p> tags for text content
            paragraphs = body.find_all('p')
            # Getting the text from each <p> tag
            text_content=[]
            for para in paragraphs:
                text_content.append(para.get_text(strip=True))

            # Audio Example:
            # <figure class="wp-block-audio">
            # <audio controls="" src="https://vot.org/wp-content/uploads/2024/08/Gangtok-TYC-54-Working-Committee-Meet-Concludes.mp3">
            # </audio></figure>
            # Find the audio tag and get its src attribute
            audio = body.find('figure', class_='wp-block-audio')
            audio_tag = audio.find('audio')
            if audio_tag:
                audio_src = audio_tag.get('src')
            else:
                audio_src = "No audio tag found"
            body_text = {
                "Audio": audio_src,
                "Text": text_content
            }
        
        else:
            body_text = "Body content not found"
        
        return {
            'title': title_text,
            'body': body_text,
            'meta_data': meta_data
        }
    except requests.RequestException as e:
        return {
            'title': "Error",
            'body': f"An error occurred while fetching the article: {str(e)}",
            'meta_data': {'URL': url}
        }

# Usage
url = "https://vot.org/%e0%bd%96%e0%bd%bc%e0%bd%91%e0%bc%8b%e0%bd%80%e0%be%b1%e0%bd%b2%e0%bc%8b%e0%bd%82%e0%bd%9e%e0%bd%bc%e0%bd%93%e0%bc%8b%e0%bd%93%e0%bd%b4%e0%bc%8b%e0%bd%a3%e0%be%b7%e0%bd%93%e0%bc%8b%e0%bd%9a-20/"
article = scrape_vot_article(url)


article

{'title': 'བོད་ཀྱི་གཞོན་ནུ་ལྷན་ཚོགས་ཀྱི་འགན་འཛིན་ལྷན་ཚོགས་ཐེངས་ ༥༤ པ་མཇུག་བསྒྲིལ་འདུག',
 'body': {'Audio': 'https://vot.org/wp-content/uploads/2024/08/Gangtok-TYC-54-Working-Committee-Meet-Concludes.mp3',
  'Text': ['ཁ་སྔོན་ཕྱི་ཚེས་ ༥ ནས་འགོ་འཛུགས་བྱུང་བའི་བོད་ཀྱི་གཞོན་ནུ་ལྷན་ཚོགས་ཀྱི་འགན་འཛིན་ལྷན་ཚོགས་ཐེངས་ ༥༤ དེ་ཕྱི་ཚེས་ ༡༠ ཉིན་མཇུག་བསྒྲིལ་ཡོད་པ་དང་།\xa0\xa0 བོད་ཀྱི་གཞོན་ནུ་ལྷན་ཚོགས་ཀྱི་ངོས་ནས་ད་རེས་ཀྱི་འགན་འཛིན་ལྷན་ཚོགས་འདིའི་བརྒྱུད།\xa0 རྒྱ་ནག་གཞུང་གིས་བོད་མིའི་ངོ་བོ་རྩ་མེད་བཟོ་བའི་སྲིད་བྱུས་འོག\xa0\xa0 བོད་ཀྱི་ཆོས་དང་རིག་གཞུང་སྐད་ཡིག་བཅས་པར་ཕྱོགས་གང་ས་ནས་དམ་དྲག་ཇེ་ཆེར་བྱེད་བཞིན་པ་དེར་ངོ་རྒོལ་གྱིས་བསྒྲགས་གཏམ་ཞིག་སྤེལ་ཡོད་པ་རེད།',
   'ད་རེས་ཀྱི་འགན་འཛིན་ལྷན་ཚོགས་ཐེངས་ ༥༤ པ་དེའི་ཐོག\xa0\xa0 རྒྱ་གར་དང་བལ་ཡུལ་བཅས་ས་གནས་ཁག་ ༣༨ ནས་ཚོགས་བཅར་བ་མི་གྲངས་\xa0 ༢༠༠ ཙམ་ཞིག་འདུ་འཛོམས་བྱུང་ཡོད་པ་དང་།\xa0 ཉིན་གྲངས་ལྔའི་རིང་ཚོགས་བཅར་བ་ཕན་ཚུན་བཀའ་བསྡུར་དང་འབྲེལ་གྲོས་ཆོད་དོན་ཚན་བརྒྱད་གཏན་འབེབས་གནང་ཐུབ་པ་མ་ཟད།\xa0 སྤྱི་མོས་གྲོས་ཆོད་ཞིག་གཏན་འབེབས་བྱུང་ཡོད་པ་དེའི་ནང་བོད་ཀྱི་སྐད་ཡིག་མི་ཉམས་རྒྱུན་འཛིན་སླད།

In [36]:
# Usage
url = "https://vot.org/vott%e0%bd%a2%e0%bd%82%e0%bd%a6%e0%bd%82%e0%bd%85%e0%bd%82%e0%bd%a2%e0%bd%84%e0%bd%a0%e0%bd%9b%e0%bd%93%e0%bd%a3%e0%bd%91/"
article = scrape_vot_article(url)



{'title': 'Error',
 'body': 'An error occurred while fetching the article: 404 Client Error: Not Found for url: https://vot.org/vott%25E0%25BD%25A2%25E0%25BD%2582%25E0%25BD%25A6%25E0%25BD%2582%25E0%25%2585%25E0%25BD%2582%25E0%25BD%25A2%25E0%25BD%2584%25E0%25BD%25A0%25E0%25BD%259B%25E0%25BD%2593%25E0%25BD%25A3%25E0%25BD%2591/',
 'meta_data': {'URL': 'https://vot.org/vott%e0%bd%a2%e0%bd%82%e0%bd%a6%e0%bd%82%e0%%85%e0%bd%82%e0%bd%a2%e0%bd%84%e0%bd%a0%e0%bd%9b%e0%bd%93%e0%bd%a3%e0%bd%91/'}}

{'title': 'ས་རཱ་མཐོ་སློབ་ཀྱི་ལོ་རྒྱུས་དང་རྩོམ་རིག་གཙུག་ལག་རབ་འབྱམས་པའི་སློབ་ཐོན་མཛད་སྒོར་རིགས་གཅིག་རྐང་འཛིན་ལ་དོ་སྣང་གནང་གལ་ཡིན་པ་གསུངས་སོང་།',
 'body': {'Audio': 'https://vot.org/wp-content/uploads/2024/08/tib_13_08_2024-1.mp3',
  'Text': ['དེ་རིང་ཕྱི་ཟླ་ ༨ ཚེས་ ༡༣ ཉིན་རིགས་ལམ་སློབ་གཉེར་ཁང་གི་ཡན་ལག་ས་རཱ་བོད་ཀྱི་མཐོ་རིམ་སློབ་གཉེར་ཁང་དབུ་བརྙེས་ཏེ་ལོ་ངོ་ ༢༦ འཁོར་བ་དང་བསྟུན། སྐབས་ལྔ་པའི་ལོ་རྒྱུས་གཙུག་ལ་རབ་འབྱམས་པའི་སློབ་མ་ ༧ དང་སྐབས་བཞི་པའི་རྩོམ་རིག་གཙུག་ལག་རབ་འབྱམས་པའི་སློབ་མ་ ༥ བཅས་ཁྱོན་སློབ་མ་ ༡༢ ལ་སློབ་ཐོན་ཕྱག་འཁྱེར་གནང་ཡོད་པ་དང་འབྲེལ། རིགས་གཅིག་རྐང་འཛིན་ལ་དོ་སྣང་ཆེན་པོ་བྱ་དགོས་ཤིང་། མ་འོངས་སྤྱི་ཚོགས་ལ་ཕན་པའི་ལས་དོན་རྒྱ་གང་ཆེ་གནང་བའི་རེ་བ་དང་སྨོན་འདུན་ཞུས་སོང་།',
   'དེ་ཡང་དེ་རིང་གི་མཛད་སྒོའི་ཐོག་སྐུ་མགྲོན་གཙོ་བོ་བོད་ཀྱི་དཔེ་མཛོད་ཁང་གི་འགན་འཛིན་དགེ་བཤེས་ལྷག་རྡོར་ལགས་དང་། རིགས་སློབ་མ་ལག་གཉིས་ཀྱི་དྲུང་ཆེ་བསྟན་འཛིན་སྟོབས་རྒྱས་ལགས། འཇར་མན་བོད་ཁང་གི་འགན་འཛིན་སྤྲུལ་སྐུ་ཐོས་བསམ་ལགས། ས་རཱ་མཐོ་སློབ་ཀྱི་སློབ་སྤྱི་བགྲེས་གཞོན་གཙོས་པའི་སློབ་དཔོན་རྣམ་པ། ལས་བྱེད། དེ་བཞིན་སློབ་གཉེར་བ་དང་སློབ་ཐོན་པའི་

## test code


In [13]:
from bs4 import BeautifulSoup

# Sample HTML content
html_content = """
<div class="td_block_wrap tdb_single_content tdi_100 td-pb-border-top td_block_template_1 td-post-content tagdiv-type" data-td-block-uid="tdi_100">
<style>.tdi_100{margin-top:0px!important}</style>
<style>.tdb_single_content{margin-bottom:0;*zoom:1}.tdb_single_content:before,.tdb_single_content:after{display:table;content:'';line-height:0}.tdb_single_content:after{clear:both}.tdb_single_content .tdb-block-inner>*:not(.wp-block-quote):not(.alignwide):not(.alignfull.wp-block-cover.has-parallax):not(.td-a-ad){margin-left:auto;margin-right:auto}.tdb_single_content a{pointer-events:auto}.tdb_single_content .td-spot-id-top_ad .tdc-placeholder-title:before{content:'Article Top Ad'!important}.tdb_single_content .td-spot-id-inline_ad0 .tdc-placeholder-title:before{content:'Article Inline Ad 1'!important}.tdb_single_content .td-spot-id-inline_ad1 .tdc-placeholder-title:before{content:'Article Inline Ad 2'!important}.tdb_single_content .td-spot-id-inline_ad2 .tdc-placeholder-title:before{content:'Article Inline Ad 3'!important}.tdb_single_content .td-spot-id-bottom_ad .tdc-placeholder-title:before{content:'Article Bottom Ad'!important}.tdb_single_content .id_top_ad,.tdb_single_content .id_bottom_ad{clear:both;margin-bottom:21px;text-align:center}.tdb_single_content .id_top_ad img,.tdb_single_content .id_bottom_ad img{margin-bottom:0}.tdb_single_content .id_top_ad .adsbygoogle,.tdb_single_content .id_bottom_ad .adsbygoogle{position:relative}.tdb_single_content .id_ad_content-horiz-left,.tdb_single_content .id_ad_content-horiz-right,.tdb_single_content .id_ad_content-horiz-center{margin-bottom:15px}.tdb_single_content .id_ad_content-horiz-left img,.tdb_single_content .id_ad_content-horiz-right img,.tdb_single_content .id_ad_content-horiz-center img{margin-bottom:0}.tdb_single_content .id_ad_content-horiz-center{text-align:center}.tdb_single_content .id_ad_content-horiz-center img{margin-right:auto;margin-left:auto}.tdb_single_content .id_ad_content-horiz-left{float:left;margin-top:9px;margin-right:21px}.tdb_single_content .id_ad_content-horiz-right{float:right;margin-top:6px;margin-left:21px}.tdb_single_content .tdc-a-ad .tdc-placeholder-title{width:300px;height:250px}.tdb_single_content .tdc-a-ad .tdc-placeholder-title:before{position:absolute;top:50%;-webkit-transform:translateY(-50%);transform:translateY(-50%);margin:auto;display:table;width:100%}.tdb_single_content .tdb-block-inner.td-fix-index{word-break:break-word}.tdi_100,.tdi_100>p,.tdi_100 .tdb-block-inner>p{font-family:MonlamOuchen2!important;font-size:20px!important;line-height:1.6!important}.tdi_100 h1{font-family:MonlamOuchen2!important}.tdi_100 h2{font-family:MonlamOuchen2!important}.tdi_100 .tdb-block-inner blockquote p{font-family:MonlamOuchen2!important;font-style:normal!important}.tdi_100 .wp-caption-text,.tdi_100 figcaption{font-family:MonlamOuchen2!important}.tdi_100 .page-nav a,.tdi_100 .page-nav span,.tdi_100 .page-nav>div{font-family:MonlamOuchen2!important}@media (max-width:767px){.tdb_single_content .id_ad_content-horiz-left,.tdb_single_content .id_ad_content-horiz-right,.tdb_single_content .id_ad_content-horiz-center{margin:0 auto 26px auto}}@media (max-width:767px){.tdb_single_content .id_ad_content-horiz-left{margin-right:0}}@media (max-width:767px){.tdb_single_content .id_ad_content-horiz-right{margin-left:0}}@media (max-width:767px){.tdb_single_content .td-a-ad{float:none;text-align:center}.tdb_single_content .td-a-ad img{margin-right:auto;margin-left:auto}.tdb_single_content .tdc-a-ad{float:none}}@media (min-width:768px) and (max-width:1018px){.tdi_100,.tdi_100>p,.tdi_100 .tdb-block-inner>p{font-size:15px!important}}</style><div class="tdb-block-inner td-fix-index">
<figure class="wp-block-audio"><audio controls="" src="https://vot.org/wp-content/uploads/2024/08/Gangtok-TYC-54-Working-Committee-Meet-Concludes.mp3"></audio></figure>
<p>ཁ་སྔོན་ཕྱི་ཚེས་ ༥ ནས་འགོ་འཛུགས་བྱུང་བའི་བོད་ཀྱི་གཞོན་ནུ་ལྷན་ཚོགས་ཀྱི་འགན་འཛིན་ལྷན་ཚོགས་ཐེངས་ ༥༤ དེ་ཕྱི་ཚེས་ ༡༠ ཉིན་མཇུག་བསྒྲིལ་ཡོད་པ་དང་།   བོད་ཀྱི་གཞོན་ནུ་ལྷན་ཚོགས་ཀྱི་ངོས་ནས་ད་རེས་ཀྱི་འགན་འཛིན་ལྷན་ཚོགས་འདིའི་བརྒྱུད།  རྒྱ་ནག་གཞུང་གིས་བོད་མིའི་ངོ་བོ་རྩ་མེད་བཟོ་བའི་སྲིད་བྱུས་འོག   བོད་ཀྱི་ཆོས་དང་རིག་གཞུང་སྐད་ཡིག་བཅས་པར་ཕྱོགས་གང་ས་ནས་དམ་དྲག་ཇེ་ཆེར་བྱེད་བཞིན་པ་དེར་ངོ་རྒོལ་གྱིས་བསྒྲགས་གཏམ་ཞིག་སྤེལ་ཡོད་པ་རེད།  </p>
<figure class="wp-block-image size-full"><img alt="TYC meeting 2024 concluded" class="wp-image-127463" decoding="async" fetchpriority="high" height="1152" sizes="(max-width: 2048px) 100vw, 2048px" src="https://vot.org/wp-content/uploads/2024/08/455184244_547361780947286_4277832480054834483_n.jpg" srcset="https://vot.org/wp-content/uploads/2024/08/455184244_547361780947286_4277832480054834483_n.jpg 2048w, https://vot.org/wp-content/uploads/2024/08/455184244_547361780947286_4277832480054834483_n-768x432.jpg 768w, https://vot.org/wp-content/uploads/2024/08/455184244_547361780947286_4277832480054834483_n-1536x864.jpg 1536w, https://vot.org/wp-content/uploads/2024/08/455184244_547361780947286_4277832480054834483_n-150x84.jpg 150w, https://vot.org/wp-content/uploads/2024/08/455184244_547361780947286_4277832480054834483_n-300x169.jpg 300w, https://vot.org/wp-content/uploads/2024/08/455184244_547361780947286_4277832480054834483_n-600x338.jpg 600w, https://vot.org/wp-content/uploads/2024/08/455184244_547361780947286_4277832480054834483_n-696x392.jpg 696w, https://vot.org/wp-content/uploads/2024/08/455184244_547361780947286_4277832480054834483_n-1392x783.jpg 1392w, https://vot.org/wp-content/uploads/2024/08/455184244_547361780947286_4277832480054834483_n-1068x601.jpg 1068w, https://vot.org/wp-content/uploads/2024/08/455184244_547361780947286_4277832480054834483_n-1920x1080.jpg 1920w, https://vot.org/wp-content/uploads/2024/08/455184244_547361780947286_4277832480054834483_n-747x420.jpg 747w, https://vot.org/wp-content/uploads/2024/08/455184244_547361780947286_4277832480054834483_n-1493x840.jpg 1493w" width="2048"/></figure>
<p>ད་རེས་ཀྱི་འགན་འཛིན་ལྷན་ཚོགས་ཐེངས་ ༥༤ པ་དེའི་ཐོག   རྒྱ་གར་དང་བལ་ཡུལ་བཅས་ས་གནས་ཁག་ ༣༨ ནས་ཚོགས་བཅར་བ་མི་གྲངས་  ༢༠༠ ཙམ་ཞིག་འདུ་འཛོམས་བྱུང་ཡོད་པ་དང་།  ཉིན་གྲངས་ལྔའི་རིང་ཚོགས་བཅར་བ་ཕན་ཚུན་བཀའ་བསྡུར་དང་འབྲེལ་གྲོས་ཆོད་དོན་ཚན་བརྒྱད་གཏན་འབེབས་གནང་ཐུབ་པ་མ་ཟད།  སྤྱི་མོས་གྲོས་ཆོད་ཞིག་གཏན་འབེབས་བྱུང་ཡོད་པ་དེའི་ནང་བོད་ཀྱི་སྐད་ཡིག་མི་ཉམས་རྒྱུན་འཛིན་སླད།  བོད་པའི་ཚོང་ཁང་དང་། ཟ་ཁང་།   མགྲོན་ཁང་།  འགྲིམ་འགྲུལ་ཁང་སོགས་ཀྱི་སྒོ་བྱང་ཐོག་ངེས་པར་དུ་བོད་ཡིག་འབྲི་འགོད་ཡོང་ཐབས་དང་།   ལྷག་པར་གཞོན་ནུའི་སྒྲིག་འཛུགས་ཁྱབ་ཁོངས་ནས་ཕྱིར་འཐེན་བྱས་པའི་སྦེལ་ཀོབ་དང་ཧོན་སུར།  ཀོ་ལི་གྷལ་བཅས་ཀྱི་གཞོན་ནུའི་མིང་འཛིན་པ་ཚོས་གཞོན་ནུའི་མཚན་བྱང་བཀོལ་སྤྱོད་མི་ཆོག་པ་བཅས་ཀྱི་གྲོས་ཆོད་ཀྱང་གཏན་འབེབས་བྱུང་ཡོད་པ་རེད། </p>
<figure class="wp-block-image size-full"><img alt="TYC meeting 2024 concluded" class="wp-image-127465" decoding="async" height="1152" sizes="(max-width: 2048px) 100vw, 2048px" src="https://vot.org/wp-content/uploads/2024/08/455277083_547360910947373_3766711190846205726_n.jpg" srcset="https://vot.org/wp-content/uploads/2024/08/455277083_547360910947373_3766711190846205726_n.jpg 2048w, https://vot.org/wp-content/uploads/2024/08/455277083_547360910947373_3766711190846205726_n-768x432.jpg 768w, https://vot.org/wp-content/uploads/2024/08/455277083_547360910947373_3766711190846205726_n-1536x864.jpg 1536w, https://vot.org/wp-content/uploads/2024/08/455277083_547360910947373_3766711190846205726_n-150x84.jpg 150w, https://vot.org/wp-content/uploads/2024/08/455277083_547360910947373_3766711190846205726_n-300x169.jpg 300w, https://vot.org/wp-content/uploads/2024/08/455277083_547360910947373_3766711190846205726_n-600x338.jpg 600w, https://vot.org/wp-content/uploads/2024/08/455277083_547360910947373_3766711190846205726_n-696x392.jpg 696w, https://vot.org/wp-content/uploads/2024/08/455277083_547360910947373_3766711190846205726_n-1392x783.jpg 1392w, https://vot.org/wp-content/uploads/2024/08/455277083_547360910947373_3766711190846205726_n-1068x601.jpg 1068w, https://vot.org/wp-content/uploads/2024/08/455277083_547360910947373_3766711190846205726_n-1920x1080.jpg 1920w, https://vot.org/wp-content/uploads/2024/08/455277083_547360910947373_3766711190846205726_n-747x420.jpg 747w, https://vot.org/wp-content/uploads/2024/08/455277083_547360910947373_3766711190846205726_n-1493x840.jpg 1493w" width="2048"/></figure>
<p>བོད་ཀྱི་གཞོན་ནུ་ལྷན་ཚོགས་ཀྱི་སྤྱི་ཁྱབ་ཚོགས་གཙོ་མགོན་པོ་དོན་གྲུབ་ལགས་ཀྱིས།   ད་རེས་ཀྱི་འགན་འཛིན་ལྷན་ཚོགས་ཐེངས་ ༥༤ པ་དེ་སྒང་ཏོག་ས་གནས་ཀྱི་གཞོན་ནུ་ལྷན་ཚོགས་ངོས་ནས་གོ་སྒྲིག་ཞུས་པ་ཞིག་ཡིན་པ་དང་།   ལྷན་ཚོགས་དེ་ཚགས་ཚུད་ཡོང་ཐབས་སུ་ས་གནས་དབུས་རྒྱུན་ཟུར་པ་དང་།  མངའ་སྡེའི་གཞུང་གི་ངོས་ནས་ཀྱང་མཐུན་འགྱུར་གང་འོས་གནང་ཡོད་པ་བཅས་འགྲེལ་བརྗོད་གནང་འདུག<a></a></p>
</div></div>
"""

# Parse the HTML
soup = BeautifulSoup(html_content, 'html.parser')

paragraphs = soup.find_all('p')

# Getting the text from each <p> tag
text_content=[]
for para in paragraphs:
    text_content.append(para.get_text(strip=True))
text_content          

['ཁ་སྔོན་ཕྱི་ཚེས་ ༥ ནས་འགོ་འཛུགས་བྱུང་བའི་བོད་ཀྱི་གཞོན་ནུ་ལྷན་ཚོགས་ཀྱི་འགན་འཛིན་ལྷན་ཚོགས་ཐེངས་ ༥༤ དེ་ཕྱི་ཚེས་ ༡༠ ཉིན་མཇུག་བསྒྲིལ་ཡོད་པ་དང་།   བོད་ཀྱི་གཞོན་ནུ་ལྷན་ཚོགས་ཀྱི་ངོས་ནས་ད་རེས་ཀྱི་འགན་འཛིན་ལྷན་ཚོགས་འདིའི་བརྒྱུད།  རྒྱ་ནག་གཞུང་གིས་བོད་མིའི་ངོ་བོ་རྩ་མེད་བཟོ་བའི་སྲིད་བྱུས་འོག   བོད་ཀྱི་ཆོས་དང་རིག་གཞུང་སྐད་ཡིག་བཅས་པར་ཕྱོགས་གང་ས་ནས་དམ་དྲག་ཇེ་ཆེར་བྱེད་བཞིན་པ་དེར་ངོ་རྒོལ་གྱིས་བསྒྲགས་གཏམ་ཞིག་སྤེལ་ཡོད་པ་རེད།',
 'ད་རེས་ཀྱི་འགན་འཛིན་ལྷན་ཚོགས་ཐེངས་ ༥༤ པ་དེའི་ཐོག   རྒྱ་གར་དང་བལ་ཡུལ་བཅས་ས་གནས་ཁག་ ༣༨ ནས་ཚོགས་བཅར་བ་མི་གྲངས་  ༢༠༠ ཙམ་ཞིག་འདུ་འཛོམས་བྱུང་ཡོད་པ་དང་།  ཉིན་གྲངས་ལྔའི་རིང་ཚོགས་བཅར་བ་ཕན་ཚུན་བཀའ་བསྡུར་དང་འབྲེལ་གྲོས་ཆོད་དོན་ཚན་བརྒྱད་གཏན་འབེབས་གནང་ཐུབ་པ་མ་ཟད།  སྤྱི་མོས་གྲོས་ཆོད་ཞིག་གཏན་འབེབས་བྱུང་ཡོད་པ་དེའི་ནང་བོད་ཀྱི་སྐད་ཡིག་མི་ཉམས་རྒྱུན་འཛིན་སླད།  བོད་པའི་ཚོང་ཁང་དང་། ཟ་ཁང་།   མགྲོན་ཁང་།  འགྲིམ་འགྲུལ་ཁང་སོགས་ཀྱི་སྒོ་བྱང་ཐོག་ངེས་པར་དུ་བོད་ཡིག་འབྲི་འགོད་ཡོང་ཐབས་དང་།   ལྷག་པར་གཞོན་ནུའི་སྒྲིག་འཛུགས་ཁྱབ་ཁོངས་ནས་ཕྱིར་འཐེན་བྱས་པའི་སྦེལ་ཀོབ་དང་ཧོན་སུར།  ཀོ་ལི་གྷལ་བཅས་ཀྱི་གཞོན་ནུའི་མིང་འཛིན་པ་ཚོས་ག

In [49]:
lsit_num = [1,2,3,54,4,5,68,5,1,5,]
for each in lsit_num:
    if each>50:
        print(each)

54
68


In [16]:
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

response = requests.get(url, headers=headers)
response.raise_for_status()
soup = BeautifulSoup(response.content, 'html.parser')


None


In [48]:

class_name = "vc_column_inner tdi_99 wpb_column vc_column_container tdc-inner-column td-pb-span8"
meta_data_body = soup.find('div', class_=class_name)
print(meta_data_body)

<div class="vc_column_inner tdi_99 wpb_column vc_column_container tdc-inner-column td-pb-span8">
<style scoped="">.tdi_99{vertical-align:baseline}.tdi_99 .vc_column-inner>.wpb_wrapper,.tdi_99 .vc_column-inner>.wpb_wrapper .tdc-elements{display:block}.tdi_99 .vc_column-inner>.wpb_wrapper .tdc-elements{width:100%}@media (min-width:768px) and (max-width:1018px){.tdi_99{width:100%!important}}</style><div class="vc_column-inner"><div class="wpb_wrapper"><div class="td_block_wrap tdb_single_content tdi_100 td-pb-border-top td_block_template_1 td-post-content tagdiv-type" data-td-block-uid="tdi_100">
<style>.tdi_100{margin-top:0px!important}</style>
<style>.tdb_single_content{margin-bottom:0;*zoom:1}.tdb_single_content:before,.tdb_single_content:after{display:table;content:'';line-height:0}.tdb_single_content:after{clear:both}.tdb_single_content .tdb-block-inner>*:not(.wp-block-quote):not(.alignwide):not(.alignfull.wp-block-cover.has-parallax):not(.td-a-ad){margin-left:auto;margin-right:auto}

In [42]:
author = meta_data_body.find('a', class_="tdb-author-name")
print(author)

<a class="tdb-author-name" href="https://vot.org/author/gurbumch_editor/">vot.org</a>


In [43]:
author.get_text()

'vot.org'

In [44]:
date_time = meta_data_body.find('time', class_="entry-date updated td-module-date").get_text()
date_time

'13/08/2024'

## title extraction

In [39]:
url = "https://vot.org/%e0%bd%96%e0%bd%bc%e0%bd%91%e0%bc%8b%e0%bd%80%e0%be%b1%e0%bd%b2%e0%bc%8b%e0%bd%82%e0%bd%9e%e0%bd%bc%e0%bd%93%e0%bc%8b%e0%bd%93%e0%bd%b4%e0%bc%8b%e0%bd%a3%e0%be%b7%e0%bd%93%e0%bc%8b%e0%bd%9a-20/"


response = requests.get(url, headers=headers)
response.raise_for_status()
soup = BeautifulSoup(response.content, 'html.parser')



In [40]:

# Extract title
title = soup.find('h1', class_='tdb-title-text')
title

<h1 class="tdb-title-text">བོད་ཀྱི་གཞོན་ནུ་ལྷན་ཚོགས་ཀྱི་འགན་འཛིན་ལྷན་ཚོགས་ཐེངས་ ༥༤ པ་མཇུག་བསྒྲིལ་འདུག</h1>

In [ ]:

title_text = title.text.strip() if title else "Title not found"

In [41]:
title.text

'བོད་ཀྱི་གཞོན་ནུ་ལྷན་ཚོགས་ཀྱི་འགན་འཛིན་ལྷན་ཚོགས་ཐེངས་ ༥༤ པ་མཇུག་བསྒྲིལ་འདུག'

In [1]:
html_content = """
<ul class="tdb-tags">
<li><span>Tags</span>
</li><li><a href="https://vot.org/tag/%e0%bd%82%e0%bd%a6%e0%bd%a2%e0%bc%8b%e0%bd%a0%e0%bd%82%e0%be%b1%e0%bd%b4%e0%bd%a2%e0%bc%8d/">གསར་འགྱུར།</a></li>
<li><a href="https://vot.org/tag/%e0%bd%96%e0%bd%99%e0%bd%93%e0%bc%8b%e0%bd%96%e0%be%b1%e0%bd%bc%e0%bd%a3%e0%bc%8d/">བཙན་བྱོལ།</a></li></ul>
</div>

"""

In [5]:

# Parse the HTML
meta_data_body = BeautifulSoup(html_content, 'html.parser')


['གསར་འགྱུར།', 'བཙན་བྱོལ།']

In [21]:
url = "https://vot.org/category/%e0%bc%b8%e0%bd%82%e0%bd%bc%e0%bd%84%e0%bc%8b%e0%bd%a6%e0%bc%8b%e0%bd%98%e0%bd%86%e0%bd%bc%e0%bd%82/"
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
response = requests.get(url, headers=headers)
response.raise_for_status()
soup = BeautifulSoup(response.content, 'html.parser')

In [29]:

all_article = soup.find("div", class_="td_block_inner tdb-block-inner td-fix-index")
article_links = all_article.find_all("a", class_="td-image-wrap")
all_links = []
for each_link in article_links:
    all_links.append(each_link.get("href"))
all_links

['https://vot.org/%e0%bc%b8%e0%bd%82%e0%bd%bc%e0%bd%84%e0%bc%8b%e0%bd%a6%e0%bc%8b%e0%bd%98%e0%bd%86%e0%bd%bc%e0%bd%82%e0%bc%8b%e0%bd%82%e0%bd%b2%e0%bd%a6%e0%bc%8b-swiss-%e0%bd%91%e0%bd%84%e0%bc%8b-liechtenstein/',
 'https://vot.org/%e0%bd%a8%e0%bc%8b%e0%bd%a2%e0%bd%b2%e0%bd%a0%e0%bd%b2%e0%bc%8b%e0%bd%93%e0%bd%b2%e0%bd%a0%e0%bd%b4%e0%bc%8b%e0%bd%a1%e0%bd%bc%e0%bd%82%e0%bc%8b%e0%bd%82%e0%bd%b2%e0%bc%8b-ubs-arena-%e0%bd%a2/',
 'https://vot.org/vot%e0%bd%82%e0%bd%84%e0%bd%a6%e0%bd%98%e0%bd%86%e0%bd%82%e0%bd%a3%e0%bd%96%e0%bd%a2%e0%bd%93%e0%bd%96%e0%bd%9e-2/',
 'https://vot.org/wisdom-of-happiness-%e0%bd%9e%e0%bd%ba%e0%bd%a6%e0%bc%8b%e0%bc%b8%e0%bd%82%e0%bd%bc%e0%bd%84%e0%bc%8b%e0%bd%a6%e0%bc%8b%e0%bd%98%e0%bd%86%e0%bd%bc%e0%bd%82%e0%bc%8b%e0%bd%91%e0%bd%84%e0%bc%8b%e0%bd%a0/',
 'https://vot.org/vott%e0%bd%82%e0%bd%84%e0%bd%a6%e0%bd%98%e0%bd%86%e0%bd%82%e0%bd%82%e0%bd%a6%e0%bd%80%e0%bd%a2/',
 'https://vot.org/vott%e0%bd%82%e0%bd%84%e0%bd%a6%e0%bd%98%e0%bd%86%e0%bd%82%e0%bd%82%e0%bd%a6%e0%bd

In [1]:
import requests
from bs4 import BeautifulSoup
import json

def scrape_tt_article(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        # Make the request to the URL
        response = requests.get(url, headers=headers)
        response.raise_for_status()  
        
        # Parse the page content with BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract title
        title = soup.find('h3')
        title_text = title.get_text(strip=True) if title else "Title not found"

        # Locate the main content container
        body = soup.find('div', class_='jeg_post_excerpt') # content-inner
        
        # Initialize variables for content
        text_content = []
        author = "Author not found"
        date = "Date not found"
        audio_src = "No audio found"

        if body:
            # Extract all <span> tags within the main content container
            spans = body.find_all('span')
            
            # Iterate through the spans and extract text
            for span in spans:
                text_content.append(span.get_text(strip=True))

            # Extract author and date from meta-data if available
            meta_data = soup.find('div', class_='jeg_post_meta')
            if meta_data:
                author_tag = meta_data.find('span', class_='by')
                date_tag = meta_data.find('time', class_='entry-date')
                audio_tag = body.find('figure', class_='wp-block-audio')
                
                author = author_tag.get_text(strip=True) if author_tag else "Author not found"
                date = date_tag.get_text(strip=True) if date_tag else "Date not found"
                
                if audio_tag:
                    audio = audio_tag.find('audio')
                    audio_src = audio.get('src') if audio else "No audio source found"
        
        # Return the structured result
        return {
            'title': title_text,
            'text': '\n'.join(text_content) if text_content else "No text content found",
            'meta_data': {
                'author': author,
                'date': date,
                'audio': audio_src
            }
        }
    
    except requests.RequestException as e:
        return {
            'title': "Error",
            'text': f"An error occurred while fetching the article: {str(e)}",
            'meta_data': {'author': "Unknown", 'date': "Unknown", 'audio': "None"}
        }

def scrape_multiple_articles(base_url, num_articles):
    articles = []
    for i in range(1, num_articles + 1):
        # Generate the URL for each article (assuming a pattern in URL)
        url = f"{base_url}/{i}/"
        print(f"Scraping {url}...")
        article = scrape_tt_article(url)
        articles.append(article)
    
    # Saving to a JSON file
    with open('articles.json', 'w', encoding='utf-8') as f:
        json.dump(articles, f, ensure_ascii=False, indent=4)

    print("All article data has been saved to 'articles.json'")



In [2]:

# Usage
base_url = "https://tibettimes.net/category/news/dalai-lama/page/"
num_articles = 2  # Specify the number of articles to scrape
scrape_multiple_articles(base_url, num_articles)

Scraping https://tibettimes.net/category/news/dalai-lama/page//1/...
Scraping https://tibettimes.net/category/news/dalai-lama/page//2/...
All article data has been saved to 'articles.json'
